In [14]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
import time
import random
import requests

In [53]:
def league_standings(url, season):
    requests_header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'     
    }
    
    #source
    source = requests.get(url, requests_header).text
    soup = BeautifulSoup(source, 'html.parser')
    
    #column names
    columnlist = []
    
    table = soup.find('table', class_ = 'stats_table sortable min_width force_mobilize')
    
    titles_only = table.find('thead')
    for columnname in titles_only.find_all('th'):
        columnlist.append(columnname.text)
    
    #table content
    content = []
    
    body = table.find('tbody')
    for contentbody in body.find_all(['th', 'td']):
        content.append(contentbody.text.replace(' ', ''))
    
    def flat_to_df(flatlist, num_columns, column_names=None):
        table_rows = [flatlist[i:i + num_columns] for i in range(0, len(flatlist), num_columns)]
        df = pd.DataFrame(table_rows, columns=column_names)
        return df
    
    if len(content) % len(columnlist) != 0:
        return None
    
    num_cols = len(columnlist)
    col_names = columnlist
    df_data = flat_to_df(content, num_cols, col_names)
    df_data.insert(0, 'Season', season)
    return df_data


per_length = 6

year_1 = 2023
year_2= 2024

full_league_standings = []
for i in range(per_length):   
    url = f'https://fbref.com/en/comps/9/{year_1}-{year_2}/{year_1}-{year_2}-Premier-League-Stats'
    con = league_standings(url, f'{year_1}-{year_2}')
    print(f'Scraping Season: {year_1}-{year_2}')
    full_league_standings.append(con)
    sleep_duration = random.uniform(5, 15)
    time.sleep(sleep_duration)
    year_1 -= 1
    year_2 -= 1

result = pd.concat(full_league_standings, ignore_index=True)
result.to_csv('/Users/ogizelenovic/Downloads/final_league_scrape.csv', encoding='utf-8-sig', index=False)

Scraping Season: 2023-2024
Scraping Season: 2022-2023
Scraping Season: 2021-2022
Scraping Season: 2020-2021
Scraping Season: 2019-2020
Scraping Season: 2018-2019
